In [1]:
import json
import random
import string
import time
import uuid

In [2]:
number_of_insurance_quotes = 100
insurance_quote_duplicates = 2

number_of_policies = 100
number_of_policy_updates = 5

customers_number = 10

# Insurance Quote Requests and Policy Events
Generates:
- 100 unique InsuranceQuoteRequest events
- 200 duplicate InsuranceQuoteRequest events (2 duplicates per insurance quote)
- 100 unique PolicyCreated events (one per insurance quote)
- 500 PolicyUpdated events (5 updates per policy). 


In [3]:
customer_ids = []
for i in range(customers_number):
    customer_ids += [''.join(random.choices(string.ascii_lowercase + string.digits, k=10))]

In [4]:
insurance_quote_ids = []
for i in range(number_of_insurance_quotes):
    insurance_quote_ids.append(i+1)

In [5]:
with open('insurance-quote.json', 'r') as file:
    insurance_quote_json = json.loads(file.read())

with open('policy-created.json', 'r') as file:
    policy_created_json = json.loads(file.read())

with open('update.json', 'r') as file:
    policy_updated_json = json.loads(file.read())
    
def jsons_with_id(insurance_quote, policy_created, policy_updated, i):
    insurance_quote_request_id = insurance_quote_ids[i]
    new_insurance_quote_request = insurance_quote.copy()
    customer_id = customer_ids[random.randint(0, customers_number - 1)]

    ts = int(time.time())
    
    new_insurance_quote_request['insuranceQuoteRequestDto']['id'] = insurance_quote_request_id
    new_insurance_quote_request['insuranceQuoteRequestDto']['customerInfo']['customerId'] = customer_id
    new_insurance_quote_request['date'] = ts

    new_policy_created = policy_created.copy() 
    new_policy_created['insuranceQuoteRequestId'] = insurance_quote_request_id
    policy_id =  ''.join(random.choices(string.ascii_lowercase, k=10))
    new_policy_created['policyId'] = policy_id

    new_policy_updated = policy_updated.copy()
    new_policy_updated['policy']['policyId'] = policy_id
    new_policy_updated['customer']['customerId'] = customer_id
    new_policy_updated['date'] = ts

    return new_insurance_quote_request, new_policy_created, new_policy_updated

In [6]:
all_events = []
for i in range(number_of_insurance_quotes):
    insurance_quote, policy_created, policy_updated = jsons_with_id(insurance_quote_json, policy_created_json, policy_updated_json, i)
    insurance_quotes = [insurance_quote] + [insurance_quote] * insurance_quote_duplicates
    policy_created = [policy_created]
    policy_updates = [policy_updated] * number_of_policy_updates
    all_events += list(map(lambda x: json.dumps(x), insurance_quotes)) + list(map(lambda x: json.dumps(x), policy_created)) +  list(map(lambda x: json.dumps(x), policy_updates))
    
random.shuffle(all_events)

In [7]:
with open('insurance-quote-events.txt', 'w') as file:
    for event in all_events:
        event_json = json.loads(event)

        if 'insuranceQuoteRequestDto' in event_json:
            event_id = event_json['insuranceQuoteRequestDto']['id']
            file.write(str(uuid.uuid4()))
            file.write(" ")
            file.write(str(event_id))
            file.write(" ")
            file.write(event)
            file.write('\n')

with open('policy-created-events.txt', 'w') as file:
    for event in all_events:
        event_json = json.loads(event)

        if 'policyId' in event_json:
            event_id = event_json['policyId']
            file.write(str(uuid.uuid4()))
            file.write(" ")
            file.write(str(event_id))
            file.write(" ")
            file.write(event)
            file.write('\n')

with open('policy-updated-events.txt', 'w') as file:
    for event in all_events:
        event_json = json.loads(event)

        if 'policy' in event_json:
            event_id = event_json['policy']['policyId']
            file.write(str(uuid.uuid4()))
            file.write(" ")
            file.write(str(event_id))
            file.write(" ")
            file.write(event)
            file.write('\n')

In [8]:
all_events[0]

'{"kind": "UpdatePolicyEvent", "originator": "PolicyManagementBackend", "date": 1661041150, "customer": {"customerId": "nff6ocivon", "firstname": "Max", "lastname": "Mustermann", "birthday": 631148400000, "streetAddress": "Oberseestrasse 10", "postalCode": "8640", "city": "Rapperswil", "email": "admin@example.com", "phoneNumber": "055 222 4111", "moveHistory": [], "links": [{"rel": "self", "href": "http://localhost:8110/customers/rgpp0wkpec?fields="}, {"rel": "address.change", "href": "http://localhost:8110/customers/rgpp0wkpec/address"}]}, "policy": {"policyId": "jredncyqvh", "customer": "rgpp0wkpec", "creationDate": 1618948272375, "policyPeriod": {"startDate": 1618956000000, "endDate": 1619215200000}, "policyType": "Life Insurance", "deductible": {"amount": 100, "currency": "CHF"}, "policyLimit": {"amount": 100000, "currency": "CHF"}, "insurancePremium": {"amount": 1000, "currency": "CHF"}, "insuringAgreement": {"agreementItems": []}, "links": [], "_expandable": ["customer"]}}'